In [ ]:
import numpy as np
import time

from pyquaternion import Quaternion
from reachy2_sdk import ReachySDK
import FramesViewer.utils as fv_utils

In [ ]:
real_reachy = ReachySDK(host='192.168.1.42')

time.sleep(1.0)

real_reachy.l_arm.turn_on()

right_start_pose = np.array([
    [0.0, 0.0, -1.0, 0.20],
    [0.0, 1.0, 0.0, -0.24],
    [1.0, 0.0, 0.0, -0.23],
    [0.0, 0.0, 0.0, 1.0],
])
left_start_pose = np.array([
    [0.0, 0.0, -1.0, 0.20],
    [0.0, 1.0, 0.0, 0.24],
    [1.0, 0.0, 0.0, -0.23],
    [0.0, 0.0, 0.0, 1.0],
])

def compute_l2_distance(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix[:-1, 3] - goal_pose[:-1, 3])

In [ ]:
def compute_rotation_distance(fk_matrix, goal_pose):
    q1 = Quaternion(matrix=fk_matrix[:3, :3])
    Q, R = np.linalg.qr(goal_pose[:3, :3])
    q2 = Quaternion(matrix=Q)
    return Quaternion.distance(q1, q2)

def execute_grasp(grasp_pose, duration: float, left: bool = False) -> bool:
    print("Executing grasp")
    pregrasp_pose = grasp_pose.copy()
    pregrasp_pose = fv_utils.translateInSelf(pregrasp_pose, [0, 0, 0.1])

    if np.linalg.norm(grasp_pose[:3, 3]) > 1.0 or grasp_pose[:3, 3][0] < 0.0:  # safety check
        raise ValueError("Grasp pose is too far away (norm > 1.0)")

    if left:
        arm = real_reachy.l_arm
    else:
        arm = real_reachyr_arm

    arm.gripper.open()
    goto_id = arm.goto_from_matrix(target=pregrasp_pose, duration=duration)

    if goto_id.id == -1:
        return False

    while not real_reachy.is_move_finished(goto_id):
        time.sleep(0.1)

    print("Going to grasp pose")
    goto_id = arm.goto_from_matrix(target=grasp_pose, duration=duration)

    if goto_id.id == -1:
        print("Goto ID is -1")
        return False

    while not real_reachy.is_move_finished(goto_id):
        time.sleep(0.1)

    print("Closing gripper")
    arm.gripper.close()

    time.sleep(2.0)
    print(f'l2 distance (in meters): {compute_l2_distance(real_reachy.l_arm.forward_kinematics(), grasp_pose)}')
    time.sleep(0.5)

    lift_pose = grasp_pose.copy()
    lift_pose[:3, 3] += np.array([0, 0, 0.20])
    goto_id = arm.goto_from_matrix(target=lift_pose, duration=duration)
    if goto_id.id == -1:
        print("Goto ID is -1")
        return False

    while not real_reachy.is_move_finished(goto_id):
        time.sleep(0.1)

    return True

In [ ]:
# Place the grasp pose copied from Grasp_poses_visualization_Reachy2 notebook here

grasp_pose = np.array([[    0.29753,     0.83017,    -0.47149,      0.2641],
       [  -0.082458,     0.51435,     0.85361,    0.085851],
       [    0.95115,    -0.21509,     0.22149,    -0.32163],
       [          0,           0,           0,           1]])

In [ ]:
execute_grasp(grasp_pose, 4.0, left=True)

In [ ]:
real_reachy.l_arm.gripper.open()
real_reachy.l_arm.goto_from_matrix(left_start_pose, duration=4.0)

In [ ]:
real_reachy.turn_off()